# RECIPE SCRAPER 

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

## STEP 1: Get all links

In [ ]:
# We don't need the navigation links
def not_nav(href):
    return href and not re.compile("category").search(href)

# Get the recipes
def get_recipe_links(url):
    all_links = []
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    recipes = soup.find("div", "recipe-archive")
    recipe_links = recipes.find_all(href=not_nav)
    for link in recipe_links:
        all_links.append(link.get('href'))
    return all_links

all_links_from_site = []
for num in range(0,184):
    links_from_page = get_recipe_links("https://www.halfbakedharvest.com/category/recipes/page/" + str(num))
    all_links_from_site.extend(links_from_page)

df = pd.DataFrame({'links': all_links_from_site})
df.to_csv('halfbakedharvest_links.csv')

## STEP 2: Iterate through all the links to get the recipe ID 
#### We need this for ingredient scraping

In [ ]:
def get_recipe_id(link):
    try:
        r = requests.get(link)
        soup = BeautifulSoup(r.content, 'html.parser')
        recipe_id = soup.find_all("div", {"class": "wprm-recipe-container"})[0]['data-recipe-id']

        url = "https://www.halfbakedharvest.com/wprm_print/" + recipe_id
        return url
    except:
        print(link)
        return('no data')
    
df['recipe_id_url'] = df.apply(lambda x: get_recipe_id(x['links']), axis=1)

## STEP 3: Get Ingredients from Recipe URL

In [ ]:
df.to_csv('halfbakedharvest_links_with_ids.csv')

In [ ]:
mega_list_of_ingredients = []
def get_recipe_ingredients(url):
    if url == "no data":
        return "no data"
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    recipe_name = soup.find_all('h2', 'wprm-recipe-name')[0].text.strip()
    ingredients = soup.find_all(itemprop="recipeIngredient")
    all_ingredients = []

    for i in ingredients:
        try:
            amount = i.find_all("span","wprm-recipe-ingredient-amount")
            amount = amount[0].text
        except:
            amount = 'no amount'
        
        try:
            unit = i.find_all("span","wprm-recipe-ingredient-unit")
            unit = unit[0].text
        except:
            unit = 'no unit'

        try:
            name = i.find_all("span","wprm-recipe-ingredient-name")
            name = name[0].text
        except:
            name = 'no name'

        all_ingredients.append({'url': url, 
                                'recipe_name': recipe_name,
                                'amount': amount, 
                                'unit': unit, 
                                'name': name})
        mega_list_of_ingredients.append({'url': url, 
                                'recipe_name': recipe_name,
                                'amount': amount, 
                                'unit': unit, 
                                'name': name})

    return all_ingredients

In [ ]:
df = pd.read_csv('halfbakedharvest_links_with_ids.csv')
df
# df['ingredients'] = df.apply(lambda x: get_recipe_ingredients(x['recipe_id_url']), axis=1)

In [ ]:
# df = df[:5]
df = pd.read_csv('halfbakedharvest_links_with_ids.csv')
df['ingredients'] = df.apply(lambda x: get_recipe_ingredients(x['recipe_id_url']), axis=1)

In [ ]:
df.to_csv('halfbakedharvest_links_with_ingredients.csv', index=False)

In [ ]:
mloi = pd.DataFrame(mega_list_of_ingredients)
mloi.to_csv('halfbakedharvest_all_ingredients.csv', index=False)

In [ ]:
df

In [ ]:
mloi

In [ ]:
df = pd.read_csv('halfbakedharvest_links_with_ingredients.csv')
df = df[:100]

In [ ]:



# def get_names(all_ingredients):
#     ingredients_obj = eval(all_ingredients)
#     names = []
#     for ingredient in ingredients_obj:
#         names.append(ingredient['name'])
#     return names


# not_compliant = ['pasta', 'bean', 'flour', 'wine', 'dough']
# def check_if_compliant(ingredient_names):
# #     print(ingredients)
#     ingredients_str = ''.join(ingredient_names)
#     for item in not_compliant:
#         if item in ingredients_str.lower():
#             return 'no'
#     else:
#         return 'yes'
    
df['names'] = df.apply(lambda x: get_names(x['ingredients']), axis=1)


In [ ]:
def get_names(all_ingredients):
#     print('getting here')
#     ingredients_obj = eval(all_ingredients)
    print(type(all_ingredients))
    
df['names'] = df.apply(lambda x: get_names(x['ingredients']), axis=1)

In [ ]:
df['ingredients'].to_dict()
type(df['ingredients'])

In [ ]:
# df = df[:5]
df.iloc[0]['ingredients'][1]

In [ ]:
df

In [ ]:
type(df.iloc[0]['ingredients'])

## 11-10-20

In [4]:
import pandas as pd
df = pd.read_csv('halfbakedharvest_links_with_ingredients.csv')

/Users/kendraryan/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [5]:
og_df = df.copy()
sm_df = df[:5]

In [6]:
eval(df.iloc[0]['ingredients'])

[{'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '2',
  'unit': 'ounces',
  'name': 'vodka'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': 'no amount',
  'unit': 'no unit',
  'name': 'juice from 1/2 of a lime'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '1/3',
  'unit': 'cup',
  'name': 'apple cider'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '1',
  'unit': 'tablespoon',
  'name': 'apple butter'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '1-2',
  'unit': 'teaspoons',
  'name': 'fresh grated ginger'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount'

In [7]:
# ATTEMPTING TO FORMAT INGREDIENTS 
for ing in eval(df.iloc[0]['ingredients']):
    name = ing['name']
    amount = '' if ing['amount'] == 'no amount' else '(' + ing['amount'] 
    unit = '' if ing['unit'] == 'no unit' else ing['unit'] + ')'
    tidy_text = '{} {} {}'.format(name, amount, unit)
    print(tidy_text)

vodka (2 ounces)
juice from 1/2 of a lime  
apple cider (1/3 cup)
apple butter (1 tablespoon)
fresh grated ginger (1-2 teaspoons)
ginger beer, for topping  
pomegranate arils and cinnamon sticks, for serving  


In [8]:
def format_ingredients(ingredients_list):
    formatted_ingredients = []
    for ing in eval(ingredients_list):
        name = ing['name']
        amount = '' if (ing['amount'] == 'no amount' and ing['unit'] == 'no unit') else '(' + ing['amount'] 
        unit = '' if ing['unit'] == 'no unit' else ing['unit'] + ')'
        tidy_text = '{} {} {}'.format(name, amount, unit)
        print(tidy_text)
        formatted_ingredients.append(tidy_text)
    return formatted_ingredients

In [10]:
sm_df

,Unnamed: 0,links,recipe_id_url,ingredients
0,0,https://www.halfbakedharvest.com/ginger-apple-...,https://www.halfbakedharvest.com/wprm_print/97228,[{'url': 'https://www.halfbakedharvest.com/wpr...
1,1,https://www.halfbakedharvest.com/brown-sugar-m...,https://www.halfbakedharvest.com/wprm_print/97182,[{'url': 'https://www.halfbakedharvest.com/wpr...
2,2,https://www.halfbakedharvest.com/skillet-chick...,https://www.halfbakedharvest.com/wprm_print/97147,[{'url': 'https://www.halfbakedharvest.com/wpr...
3,3,https://www.halfbakedharvest.com/brussels-spro...,https://www.halfbakedharvest.com/wprm_print/97107,[{'url': 'https://www.halfbakedharvest.com/wpr...
4,4,https://www.halfbakedharvest.com/cider-pumpkin...,https://www.halfbakedharvest.com/wprm_print/96340,[{'url': 'https://www.halfbakedharvest.com/wpr...


In [11]:
result = [format_ingredients(x) for x in sm_df['ingredients']]
results_flattened = [item for sublist in result for item in sublist]
results_flattened




vodka (2 ounces)
juice from 1/2 of a lime  
apple cider (1/3 cup)
apple butter (1 tablespoon)
fresh grated ginger (1-2 teaspoons)
ginger beer, for topping  
pomegranate arils and cinnamon sticks, for serving  
salted butter, at room temperature (1 sticks (1/2 cup))
packed dark brown sugar (1/2 cup)
pure maple syrup (1/4 cup)
vanilla extract (1 teaspoon)
egg, at room temperature (1 
all-purpose flour (1 1/2 cups)
baking soda (1/2 teaspoon)
kosher salt (1/2 teaspoon)
salted butter, at room temperature (4 tablespoons)
packed dark brown sugar (1/2 cup)
real maple syrup (1/2 cup)
vanilla extract (2 teaspoons)
eggs, at room temperature (2 
bourbon (1 tablespoon)
roughly chopped pecans (2 cups)
semi-sweet chocolate chips (1 - 1 1/2 cups)
extra virgin olive oil (2 tablespoons)
salted butter (8 tablespoons)
shallots, thinly sliced (2 
chopped fresh thyme (2 tablespoons)
chopped fresh sage, + 8 whole sage leaves (1 tablespoon  )
all-purpose flour (1/2 cup)
dry white wine, such as Pinot Grigio or

['vodka (2 ounces)',
 'juice from 1/2 of a lime  ',
 'apple cider (1/3 cup)',
 'apple butter (1 tablespoon)',
 'fresh grated ginger (1-2 teaspoons)',
 'ginger beer, for topping  ',
 'pomegranate arils and cinnamon sticks, for serving  ',
 'salted butter, at room temperature (1 sticks (1/2 cup))',
 'packed dark brown sugar (1/2 cup)',
 'pure maple syrup (1/4 cup)',
 'vanilla extract (1 teaspoon)',
 'egg, at room temperature (1 ',
 'all-purpose flour (1 1/2 cups)',
 'baking soda (1/2 teaspoon)',
 'kosher salt (1/2 teaspoon)',
 'salted butter, at room temperature (4 tablespoons)',
 'packed dark brown sugar (1/2 cup)',
 'real maple syrup (1/2 cup)',
 'vanilla extract (2 teaspoons)',
 'eggs, at room temperature (2 ',
 'bourbon (1 tablespoon)',
 'roughly chopped pecans (2 cups)',
 'semi-sweet chocolate chips (1 - 1 1/2 cups)',
 'extra virgin olive oil (2 tablespoons)',
 'salted butter (8 tablespoons)',
 'shallots, thinly sliced (2 ',
 'chopped fresh thyme (2 tablespoons)',
 'chopped fresh sa

In [13]:
# result = [format_ingredients(x) for x in sm_df['ingredients']]
shopping_list = {}
def add_ingredients_to_dictionary(formatted_ingredient):
    print(formatted_ingredient)
    ing_split = formatted_ingredient.split('(')
    print(ing_split)
    ingredient = ing_split[0]
    amount;
    try:
        amount = ing_split[1].split(')')[0]
    except:
        amount = ''
    print(ingredient, amount)
#     if ingredient in shopping_list:
#         shopping_list[ingredient] = shopping_list[ingredient] + 


In [34]:
# result = [format_ingredients(x) for x in sm_df['ingredients']]
shopping_list = {}
def add_ingredients_to_dictionary(formatted_ingredient):
    ing_split = formatted_ingredient.split(' (')
    try:
        ingredient = ing_split[0].strip()
        amount = ing_split[1].split(')')[0].strip()
    except:
        ingredient = formatted_ingredient.strip()
        amount = ''
#     print('ingredient', ingredient, 'amount', amount)
    if ingredient in shopping_list:
        shopping_list[ingredient] = shopping_list[ingredient] + ' + ' +amount
    else:
        shopping_list[ingredient] = amount


In [35]:
my_list = [add_ingredients_to_dictionary(x) for x in results_flattened]

In [32]:
"vodka (2 ounces)".split(' (')[0]

'vodka'

In [36]:
shopping_list

{'vodka': '2 ounces',
 'juice from 1/2 of a lime': '',
 'apple cider': '1/3 cup',
 'apple butter': '1 tablespoon',
 'fresh grated ginger': '1-2 teaspoons',
 'ginger beer, for topping': '',
 'pomegranate arils and cinnamon sticks, for serving': '',
 'salted butter, at room temperature': '1 sticks + 4 tablespoons',
 'packed dark brown sugar': '1/2 cup + 1/2 cup',
 'pure maple syrup': '1/4 cup + 2 tablespoons',
 'vanilla extract': '1 teaspoon + 2 teaspoons + 2 teaspoons',
 'egg, at room temperature': '1',
 'all-purpose flour': '1 1/2 cups + 1/2 cup + 2 cups',
 'baking soda': '1/2 teaspoon + 1/2 teaspoon',
 'kosher salt': '1/2 teaspoon + 1/2 teaspoon',
 'real maple syrup': '1/2 cup',
 'eggs, at room temperature': '2',
 'bourbon': '1 tablespoon',
 'roughly chopped pecans': '2 cups',
 'semi-sweet chocolate chips': '1 - 1 1/2 cups',
 'extra virgin olive oil': '2 tablespoons + 2 tablespoons',
 'salted butter': '8 tablespoons + 4 tablespoons',
 'shallots, thinly sliced': '2',
 'chopped fresh th

## PROBLEM: "eggs at room tempterature" & "Large eggs", "real maple syrup" and "maple syrup"

In [144]:
# result = [format_ingredients(x) for x in sm_df['ingredients']]
shopping_list = {}
def add_ingredients_to_dictionary(formatted_ingredient):
    ing_split = formatted_ingredient.split(' (')
    try:
        ingredient = ing_split[0].strip()
        try:
            sm_ingredient = ingredient.split(',')[0]
            instructions = ingredient.split(',')[1]
            amount = ing_split[1].split(')')[0].strip() + '(' +instructions+')'
        except:
            amount = ing_split[1].split(')')[0].strip()
    except:
        ingredient = formatted_ingredient.strip()
        amount = ''
#     print('ingredient', ingredient, 'amount', amount)
    if ingredient in shopping_list:
        shopping_list[ingredient] = shopping_list[ingredient] + ' + ' +amount
    else:
        shopping_list[ingredient] = amount


In [145]:
my_list = [add_ingredients_to_dictionary(x) for x in results_flattened]
my_list

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [42]:
ingredient = 'eggs, at room temperature'

In [43]:
sm_ingredient = ingredient.split(',')[0]
instructions = ingredient.split(',')[1]
amount = ing_split[1].split(')')[0].strip() + '(' +instructions+')'

NameError: name 'ing_split' is not defined

## PROBLEM: Instructions "Garlic, minced or grated"

In [49]:
for ing in eval(df.iloc[3]['ingredients']):
    name = ing['name']
    amount = '' if ing['amount'] == 'no amount' else '(' + ing['amount'] 
    unit = '' if ing['unit'] == 'no unit' else ing['unit'] + ')'
    tidy_text = '{} {} {}'.format(name, amount, unit)
    print(tidy_text)

fettuccine pasta (1 pound)
thick-cut slices bacon, chopped (4 
extra virgin olive oil (2 tablespoons)
Brussels sprouts, thinly sliced or halved, if small (3/4 pound)
kosher salt and black pepper  
salted butter (4 tablespoons)
garlic, minced or grated (3 cloves)
chopped fresh oregano (1 tablespoon)
chopped fresh sage (1 tablespoon)
crushed red pepper flakes (1  pinch)
whole milk or canned coconut milk (2 cups)
heavy cream or canned coconut milk (1/2 cup)
cream cheese, at room temperature  (2 ounces)
grated Parmesan or Pecorino Romano  (1 1/2 cups)
ground nutmeg (1/4 teaspoon)


In [59]:
for ing in eval(df.iloc[3]['ingredients']):
    tidy_text = ''
    name = ing['name']
    amount = ing['amount']
    unit = ing['unit']
    instructions = ''
    if ',' in name:
        new_name = name.split(',')[0]
        instructions = name.split(',')[1]
        name = new_name
        
    if unit == 'no unit' and amount == 'no amount' and instructions == '':
        tidy_text = name
        
    if unit == 'no unit' and amount != 'no amount' and instructions == '':
        tidy_text = '{} ({})'.format(name, amount)
        
    if unit == 'no unit' and amount != 'no amount' and instructions != '':
        tidy_text = '{} ({}, {})'.format(name, amount, instructions)    
        
    if unit != 'no unit' and amount != 'no amount' and instructions == '':
        tidy_text = '{} ({} {})'.format(name, amount, unit)

    if unit != 'no unit' and amount != 'no amount' and instructions != '':
        tidy_text = '{} ({} {}, {})'.format(name, amount, unit, instructions)
        
    print(tidy_text)
        
   

fettuccine pasta (1 pound)
thick-cut slices bacon (4,  chopped)
extra virgin olive oil (2 tablespoons)
Brussels sprouts (3/4 pound,  thinly sliced or halved)
kosher salt and black pepper
salted butter (4 tablespoons)
garlic (3 cloves,  minced or grated)
chopped fresh oregano (1 tablespoon)
chopped fresh sage (1 tablespoon)
crushed red pepper flakes (1  pinch)
whole milk or canned coconut milk (2 cups)
heavy cream or canned coconut milk (1/2 cup)
cream cheese (2 ounces,  at room temperature )
grated Parmesan or Pecorino Romano  (1 1/2 cups)
ground nutmeg (1/4 teaspoon)


In [69]:
methods = ['chopped', 'crushed', 'grated', 'ground']

def format_ingredients(ingredients_list):
    formatted_ingredients = []
    for ing in eval(ingredients_list):
        tidy_text = ''
        name = ing['name']
        amount = ing['amount']
        unit = ing['unit']
        instructions = ''
        if ',' in name:
            new_name = name.split(',')[0]
            instructions = name.split(',')[1]
            name = new_name

        for method in methods:
            if method in name:
                instructions = method
                new_name = name.replace(method + ' ', '')
                name = new_name

        if unit == 'no unit' and amount == 'no amount' and instructions == '':
            tidy_text = name

        if unit == 'no unit' and amount != 'no amount' and instructions == '':
            tidy_text = '{} ({})'.format(name, amount)

        if unit == 'no unit' and amount != 'no amount' and instructions != '':
            tidy_text = '{} ({}, {})'.format(name, amount, instructions)    

        if unit != 'no unit' and amount != 'no amount' and instructions == '':
            tidy_text = '{} ({} {})'.format(name, amount, unit)

        if unit != 'no unit' and amount != 'no amount' and instructions != '':
            tidy_text = '{} ({} {}, {})'.format(name, amount, unit, instructions)

        formatted_ingredients.append(tidy_text)
        
   

In [70]:
result = [format_ingredients(x) for x in sm_df['ingredients']]
# results_flattened = [item for sublist in result for item in sublist]
# results_flattened


In [74]:
def get_tidy_text_from_ingredient_dict(ing):
    tidy_text = ''
    name = ing['name']
    amount = ing['amount']
    unit = ing['unit']
    instructions = ''
    if ',' in name:
        new_name = name.split(',')[0]
        instructions = name.split(',')[1]
        name = new_name

    for method in methods:
        if method in name:
            instructions = method
            new_name = name.replace(method + ' ', '')
            name = new_name

    if unit == 'no unit' and amount == 'no amount' and instructions == '':
        tidy_text = name

    if unit == 'no unit' and amount != 'no amount' and instructions == '':
        tidy_text = '{} ({})'.format(name, amount)

    if unit == 'no unit' and amount != 'no amount' and instructions != '':
        tidy_text = '{} ({}, {})'.format(name, amount, instructions)    

    if unit != 'no unit' and amount != 'no amount' and instructions == '':
        tidy_text = '{} ({} {})'.format(name, amount, unit)

    if unit != 'no unit' and amount != 'no amount' and instructions != '':
        tidy_text = '{} ({} {}, {})'.format(name, amount, unit, instructions)

    return tidy_text
        

In [83]:

test = eval(df.iloc[0]['ingredients'])[0]
test2 = eval(df.iloc[0]['ingredients'])
get_tidy_text_from_ingredient_dict(test)

'vodka (2 ounces)'

In [86]:
result = [get_tidy_text_from_ingredient_dict(x) for x in test2]


In [87]:
result

['vodka (2 ounces)',
 'juice from 1/2 of a lime',
 'apple cider (1/3 cup)',
 'apple butter (1 tablespoon)',
 'fresh ginger (1-2 teaspoons, grated)',
 '',
 '']

## PROBLEM Dealing with the "ORs"

# 11-12-20

In [93]:
import pandas as pd
df = pd.read_csv('halfbakedharvest_links_with_ingredients.csv')
og_df = df.copy()

In [94]:
df = og_df[:10]

In [96]:
eval(df.iloc[0]['ingredients'])

[{'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '2',
  'unit': 'ounces',
  'name': 'vodka'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': 'no amount',
  'unit': 'no unit',
  'name': 'juice from 1/2 of a lime'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '1/3',
  'unit': 'cup',
  'name': 'apple cider'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '1',
  'unit': 'tablespoon',
  'name': 'apple butter'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '1-2',
  'unit': 'teaspoons',
  'name': 'fresh grated ginger'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount'

In [99]:
ing_df = [eval(x) for x in df['ingredients']]

In [101]:
test = pd.DataFrame(ing_df)
test

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,None,None,None,None,None,None,None
2,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,None,None,None,None,None,None
3,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,None,None,None,None,None,None,None,None
4,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,None,None,None,None,None,None,None
5,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,None,None,None,None,None,None,None,None
6,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'https://www.halfbakedharvest.com/wprm...,{'url': 'htt

[{'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '2',
  'unit': 'ounces',
  'name': 'vodka'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': 'no amount',
  'unit': 'no unit',
  'name': 'juice from 1/2 of a lime'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '1/3',
  'unit': 'cup',
  'name': 'apple cider'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '1',
  'unit': 'tablespoon',
  'name': 'apple butter'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount': '1-2',
  'unit': 'teaspoons',
  'name': 'fresh grated ginger'},
 {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
  'recipe_name': 'Ginger Apple Moscow Mule',
  'amount'

In [106]:
for ing in eval(df.iloc[3]['ingredients']):
    tidy_text = ''
    name = ing['name']
    amount = ing['amount']
    unit = ing['unit']
    instructions = ''
    if ',' in name:
        new_name = name.split(',')[0]
        instructions = name.split(',')[1]
        name = new_name
    print(instructions)


 chopped

 thinly sliced or halved


 minced or grated





 at room temperature 




In [220]:
meats = ['chicken', 'chicken broth', 'broccoli', 'bacon']
methods = ['chopped', 'roughly chopped', 'crushed', 'grated', 'ground', 'freshly ground']
to_remove = ['fresh']

def format_ingredients(ingredients_list):
    ing_dict = eval(ingredients_list)
    for ing in ing_dict:
        tidy_text = ''
        name = ing['name']
        amount = ing['amount']
        unit = ing['unit']
        instructions = ''
        simple_name = ''
        
        new_name = ''
        
        for word in to_remove:
            if word in name:
                name = name.replace(word + ' ', '')
                
                
        if ',' in name:
            new_name = name.split(',')[0]
            instructions = name.split(',')[1].strip()
            simple_name = new_name


        for method in methods:
            if method in name:
                instructions = method.strip()
                new_name = name.split(',')[0]
                new_name = new_name.replace(method + ' ', '').strip()
                print(new_name)
                simple_name = new_name
                ing['simple_name'] = simple_name
                ing['instructions'] = instructions   

        for meat in meats:
            if meat in name:
                name = name.replace(',', '')
#                 print(name)
                instructions = name.replace(meat, '').strip()
                simple_name = meat.strip() 
                ing['simple_name'] = simple_name
                ing['instructions'] = instructions                
#         print(instructions)
        ing['instructions'] = instructions
        
        
        if simple_name == '':
            simple_name = name

        ing['simple_name'] = simple_name
    return ing_dict
result = [format_ingredients(x) for x in df['ingredients']]
result

ginger
roughly pecans
pecans
thyme
sage
celery
carrots
roughly broccoli florets
broccoli florets
garlic
rosemary
thick-cut slices bacon
garlic
oregano
sage
red pepper flakes
Parmesan or Pecorino Romano
nutmeg
ginger
nutmeg
medium yellow onion
garlic
poblano peppers
chipotle peppers in adobo
cumin
cilantro
ginger
cinnamon
cinnamon
ginger
cardamom
freshly nutmeg
cloves
black pepper
yellow onion
ginger
garlic
cilantro
kale
red pepper flakes
cumin seed
parmesan cheese
garlic
sage
fennel seed
red pepper flakes


[[{'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
   'recipe_name': 'Ginger Apple Moscow Mule',
   'amount': '2',
   'unit': 'ounces',
   'name': 'vodka',
   'instructions': '',
   'simple_name': 'vodka'},
  {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
   'recipe_name': 'Ginger Apple Moscow Mule',
   'amount': 'no amount',
   'unit': 'no unit',
   'name': 'juice from 1/2 of a lime',
   'instructions': '',
   'simple_name': 'juice from 1/2 of a lime'},
  {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
   'recipe_name': 'Ginger Apple Moscow Mule',
   'amount': '1/3',
   'unit': 'cup',
   'name': 'apple cider',
   'instructions': '',
   'simple_name': 'apple cider'},
  {'url': 'https://www.halfbakedharvest.com/wprm_print/97228',
   'recipe_name': 'Ginger Apple Moscow Mule',
   'amount': '1',
   'unit': 'tablespoon',
   'name': 'apple butter',
   'instructions': '',
   'simple_name': 'apple butter'},
  {'url': 'https://www.halfbakedharvest.com/wprm

In [221]:
# result = [format_ingredients(x) for x in sm_df['ingredients']]
# result = [format_ingredients(x) for x in sm_df['ingredients']]
results_flattened = [item for sublist in result for item in sublist]
# results_flattened
shopping_list = {}
for i,list_item in enumerate(results_flattened): 
    r = list_item['simple_name']
    name = list_item['name']
    amount = list_item['amount']
    unit = list_item['unit']
    instructions = list_item['instructions']
    tidy_text = ''
    recipe_number = list_item['url'].split('/')[-1]
    
#     WITH RECIPE "NUMBER"
    if unit == 'no unit' and amount == 'no amount' and instructions == '':
        tidy_text = '1'

    if unit == 'no unit' and amount != 'no amount' and instructions == '':
        tidy_text = 'R{}-({})'.format(recipe_number, amount)

    if unit == 'no unit' and amount != 'no amount' and instructions != '':
        tidy_text = 'R{}-({}, {})'.format(recipe_number, amount, instructions)    

    if unit != 'no unit' and amount != 'no amount' and instructions == '':
        tidy_text = 'R{}-({} {})'.format(recipe_number, amount, unit)

    if unit != 'no unit' and amount != 'no amount' and instructions != '':
        tidy_text = 'R{}-({} {}, {})'.format(recipe_number, amount, unit, instructions)

#     WITHOUT RECIPE "NUMBER"
    if unit == 'no unit' and amount == 'no amount' and instructions == '':
        tidy_text = '1'

    if unit == 'no unit' and amount != 'no amount' and instructions == '':
        tidy_text = '({})'.format(amount)

    if unit == 'no unit' and amount != 'no amount' and instructions != '':
        tidy_text = '({}, {})'.format(amount, instructions)    

    if unit != 'no unit' and amount != 'no amount' and instructions == '':
        tidy_text = '({} {})'.format(amount, unit)

    if unit != 'no unit' and amount != 'no amount' and instructions != '':
        tidy_text = '({} {}, {})'.format(amount, unit, instructions)
    
    if r in shopping_list:
        shopping_list[r] = shopping_list[r] +  ', ' + tidy_text 
    else:
        shopping_list[r] = tidy_text

In [222]:
shopping_list

{'vodka': '(2 ounces)',
 'juice from 1/2 of a lime': '1',
 'apple cider': '(1/3 cup), (1/4 cup)',
 'apple butter': '(1 tablespoon)',
 'ginger': '(1-2 teaspoons, grated), (1/2 teaspoon, ground), (1 teaspoon, grated), (2 teaspoon, ground), (2 inches, grated)',
 'ginger beer': '',
 'pomegranate arils and cinnamon sticks': '',
 'salted butter': '(1 sticks (1/2 cup), at room temperature), (4 tablespoons, at room temperature), (8 tablespoons), (4 tablespoons), (8 tablespoons, at room temperature)',
 'packed dark brown sugar': '(1/2 cup), (1/2 cup), (3/4 cup), (1/2 cup)',
 'pure maple syrup': '(1/4 cup), (2 tablespoons)',
 'vanilla extract': '(1 teaspoon), (2 teaspoons), (2 teaspoons), (1 tablespoon), (2 teaspoons)',
 'egg': '(1, at room temperature)',
 'all-purpose flour': '(1 1/2 cups), (1/2 cup), (2 cups), (1 1/2 cups)',
 'baking soda': '(1/2 teaspoon), (1/2 teaspoon), (1/2 teaspoon)',
 'kosher salt': '(1/2 teaspoon), (1/2 teaspoon), (3/4 teaspoon)',
 'real maple syrup': '(1/2 cup), (1-2 t

## PROBLEM: Egg, eggs, large eggs

In [223]:
test = 'https://www.halfbakedharvest.com/wprm_print/97107'
test.split('/')[-1]

'97107'

In [224]:
test = {'url': 'https://www.halfbakedharvest.com/wprm_print/97147',
   'recipe_name': 'Skillet Chicken and Broccoli Pot Pie with Garlic Butter Phyllo Crust',
   'amount': '2-4',
   'unit': 'cloves',
   'name': 'garlic, grated',
   'simple_name': 'garlic, grated',
   'instructions': 'grated'}

test2 = {'url': 'https://www.halfbakedharvest.com/wprm_print/97107',
   'recipe_name': 'Brown Butter Brussels Sprout and Bacon Fettuccine Alfredo',
   'amount': '1',
   'unit': 'tablespoon',
   'name': 'chopped fresh sage',
   'simple_name': 'chopped sage',
   'instructions': 'chopped'}

In [225]:
# test = "{'url': 'https://www.halfbakedharvest.com/wprm_print/97147','recipe_name': 'Skillet Chicken and Broccoli Pot Pie with Garlic Butter Phyllo Crust','amount': '2-4','unit': 'cloves','name': 'garlic, grated','simple_name': 'garlic, grated','instructions': 'grated'}"


In [226]:
meats = ['chicken', 'chicken broth', 'broccoli', 'bacon']
methods = ['chopped', 'roughly chopped', 'crushed', 'grated','ground']
to_remove = ['fresh']


ing = test2

tidy_text = ''
name = ing['name']
amount = ing['amount']
unit = ing['unit']
instructions = ''
simple_name = ''


new_name = ''

if ',' in name:

    new_name = name.split(',')[0]
    instructions = name.split(',')[1].strip()
    simple_name = new_name
    ing['simple_name'] = simple_name
    ing['instructions'] = instructions   



for method in methods:
    if method in name:
        instructions = method.strip()
        new_name = name.split(',')[0]
        new_name = new_name.replace(method + ' ', '').strip()
        print(new_name)
        simple_name = new_name
        ing['simple_name'] = simple_name
        ing['instructions'] = instructions   


for meat in meats:
    if meat in name:
        name = name.replace(',', '')
#                 print(name)
        instructions = name.replace(meat, '').strip()
        simple_name = meat.strip() 
        ing['simple_name'] = simple_name
        ing['instructions'] = instructions                
#         print(instructions)
ing['instructions'] = instructions


if simple_name == '':
    simple_name = name

ing['simple_name'] = simple_name
print(ing)



fresh sage
{'url': 'https://www.halfbakedharvest.com/wprm_print/97107', 'recipe_name': 'Brown Butter Brussels Sprout and Bacon Fettuccine Alfredo', 'amount': '1', 'unit': 'tablespoon', 'name': 'chopped fresh sage', 'simple_name': 'fresh sage', 'instructions': 'chopped'}


In [2]:
 test2 = {'url': 'https://www.halfbakedharvest.com/wprm_print/97107',
   'recipe_name': 'Brown Butter Brussels Sprout and Bacon Fettuccine Alfredo',
   'amount': '1',
   'unit': 'tablespoon',
   'name': 'chopped fresh sage',
   'simple_name': 'chopped sage',
   'instructions': 'chopped'},

# 11-16-20
## GOAL: make a recipe db, and an ingredients db

In [4]:
import pandas as pd
df = pd.read_csv('halfbakedharvest_links_with_ingredients.csv')
og_df = df.copy()

/Users/kendraryan/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


1. go through each item in the array
2. grab the ingredient
3. grab the unit
4. grab the amount
5. grab the instructions

In [17]:
# lst = [j + k for j in s1 for k in s2]
df = df[df['ingredients'] != 'no data']
test = df['ingredients']
only_urls = [y['name'] for x in test for y in eval(x)]

In [18]:
only_urls

['vodka',
 'juice from 1/2 of a lime',
 'apple cider',
 'apple butter',
 'fresh grated ginger',
 'ginger beer, for topping',
 'pomegranate arils and cinnamon sticks, for serving',
 'salted butter, at room temperature',
 'packed dark brown sugar',
 'pure maple syrup',
 'vanilla extract',
 'egg, at room temperature',
 'all-purpose flour',
 'baking soda',
 'kosher salt',
 'salted butter, at room temperature',
 'packed dark brown sugar',
 'real maple syrup',
 'vanilla extract',
 'eggs, at room temperature',
 'bourbon',
 'roughly chopped pecans',
 'semi-sweet chocolate chips',
 'extra virgin olive oil',
 'salted butter',
 'shallots, thinly sliced',
 'chopped fresh thyme',
 'chopped fresh sage, + 8 whole sage leaves',
 'all-purpose flour',
 'dry white wine, such as Pinot Grigio or Sauvignon Blanc',
 'low sodium chicken broth',
 'boneless skinless chicken breast, sliced in half ',
 'kosher salt and black pepper',
 'celery,  chopped',
 'carrots, chopped',
 'roughly chopped broccoli florets',
 

In [23]:
ing_dict = [y for x in test for y in eval(x)]

In [25]:
test_df = pd.DataFrame(ing_dict)

In [27]:
test_df['recipe_id'] = test_df.apply(lambda x: x['url'].split('/')[-1], axis=1)

In [29]:
test_df.to_csv('halfbakedharvest_ingredients_only.csv', index=False)